# Задание

Дан файл HR.csv с данными по опросу уровня удовлетворенности сотрудниками работой.

Признаки:
1. satisfaction_level​- Уровень удовлетворенности работой
2. Last_evaluation -​ Время с момента последней оценки в годах
3. number_projects​- Количество проектов, выполненных за время работы
4. average_monthly_hours​- Среднее количество часов на рабочем месте в месяц
5. time_spend_company​- Стаж работы в компании в годах
6. work_accident​- Происходили ли несчастные случаи на рабочем месте с сотрудником
7. left​- уволился ли сотрудник
8. promotion_last_5years​- повышался ли сотрудник за последние пять лет
9. department -​ отдел в котором работает сотрудник
10. salary​- относительный уровень зарплаты

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

## Задание 1
Загрузите файл HR.csv в pandas dataframe

In [ ]:
main_df = pd.read_csv('HR.csv')
main_df.info()
main_df.dropna().info()

In [ ]:
main_df.head(10)
#main_df.isna().sum()

## Задание 2
Рассчитайте основные статистики для переменных (среднее,медиана,мода,мин/макс,сред.отклонение).

In [ ]:
main_df.describe()

In [ ]:
main_df.mean(numeric_only=True) #среднее

In [ ]:
main_df.median(numeric_only=True) #медиана

In [ ]:
main_df.mode(numeric_only=True) #мода

In [ ]:
main_df.min(numeric_only=True) #минимум

In [ ]:
main_df.max(numeric_only=True) #максимум

In [ ]:
main_df.std(numeric_only=True) #стандартное отклонение

## Задание 3
Рассчитайте и визуализировать корреляционную матрицу для количественных переменных.
Определите две самые скоррелированные и две наименее скоррелированные переменные.

In [ ]:
corr = main_df.corr(numeric_only=True)
corr.style.background_gradient(cmap='RdYlGn')

In [ ]:
sns.heatmap(corr) # построение тепловой карты с помощью библиотеки seaborn

In [ ]:
corr1 = corr.unstack().sort_values()
corr1[corr1 == corr1.min()] #минимальная пара

In [ ]:
corr2 = corr1[corr1 < 1.0]
corr2[corr2 == corr2.max()] #максимальная пара

## Задание 4
Рассчитайте сколько сотрудников работает в каждом департаменте.

In [ ]:
main_df['department'][main_df['left'] ==0].value_counts() #только работающие сейчас сотрудники

## Задание 5
Показать распределение сотрудников по зарплатам.

In [ ]:
main_df['salary'][main_df['left'] == 0].hist(bins=30) #только работающие сотрудники
plt.xlabel('salary')
plt.ylabel('workers')
plt.show()

## Задание 6
Показать распределение сотрудников по зарплатам в каждом департаменте по отдельности

In [ ]:
plt.figure(figsize=(10,18))
j = 1
for department in main_df['department'].unique():
    plt.subplot(5,2,j)
    main_df['salary'][(main_df['left'] == 0) & (main_df['department'] == department)].hist(bins=30)
    plt.title(department)
    #plt.xlabel('salary')
    plt.ylabel('workers')
    j += 1
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
j = 1
for department in main_df['department'].unique():
    main_df['salary'][(main_df['left'] == 0) & (main_df['department'] == department)].hist(bins=(10 + j*5))
    plt.title('Распределение зарплат')
    plt.xlabel('salary')
    plt.ylabel('workers')
    j += 1
plt.legend(main_df['department'].unique())
plt.show()

## Задание 7
Проверить гипотезу, что сотрудники с высоким окладом проводят на работе больше времени, чем сотрудники с низким окладом

In [ ]:
# H0 - сотрудники проводят примерно одинаково времени на работе
# H1 - сотрудники с высоким окладом проводят на работе больше времени
#print(main_df[main_df['salary'] == 'high']['average_montly_hours'].count()) #1237 выборок
#print(main_df[main_df['salary'] == 'low']['average_montly_hours'].count()) #7316 выборок
high_salary = main_df[main_df['salary'] == 'high']['average_montly_hours'].values
low1 = main_df[main_df['salary'] == 'low'].reset_index()
low1.head(100)
low1 = low1['average_montly_hours'].groupby(np.arange(len(low1.index))//5, axis=0).mean()
# усреднили группами по 5 значений? так как их много
print(low1.mean())
print(main_df[main_df['salary'] == 'low']['average_montly_hours'].values.mean())
# среднее близко
print(main_df[main_df['salary'] == 'high']['average_montly_hours'].values.mean())
low_salary = low1.iloc[0:1237].values # сравняли выборки
p = stats.ttest_ind(high_salary, low_salary, equal_var = True)
print(p)
if p[1] < 0.05:
    print("Отклоняем нулевую гипотезу")
else:
    print("Не отклоняем нулевую гипотезу")

## Задание 8
Рассчитать следующие показатели среди уволившихся и не уволившихся сотрудников (по отдельности):
● Доля сотрудников с повышением за последние 5 лет
● Средняя степень удовлетворенности
● Среднее количество проектов

In [ ]:
working = main_df[main_df['left'] == 0]
lefted = main_df[main_df['left'] == 1]

In [ ]:
print('Для работающих сотрудников')
print(f"Доля сотрудников с повышением за последние 5 лет: {working[working['promotion_last_5years'] == 1]['promotion_last_5years'].count() / working['promotion_last_5years'].count() *100:.3f}%")
print(f"Средняя степень удовлетворенности: {working['satisfaction_level'].mean():.2f}")
print(f"Среднее количество проектов: {working['number_project'].mean():.2f}")


In [ ]:
print('Для бывших сотрудников')
print(f"Доля сотрудников с повышением за последние 5 лет: {lefted[lefted['promotion_last_5years'] == 1]['promotion_last_5years'].count() / working['promotion_last_5years'].count() *100:.3f}%")
print(f"Средняя степень удовлетворенности: {lefted['satisfaction_level'].mean():.2f}")
print(f"Среднее количество проектов: {lefted['number_project'].mean():.2f}")

## Задание 9
Разделить данные на тестовую и обучающую выборки Построить модель LDA, предсказывающую уволился ли сотрудник на основе имеющихся факторов (кроме department и salary)
Оценить качество модели на тестовой выборки

In [ ]:
X = main_df[['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']]
y = main_df['left'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=2/3, random_state=0)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)


In [ ]:
result = pd.DataFrame([y_test, lda.predict(X_test)]).T
accuracy_score(y_test, lda.predict(X_test))

Ни изменение тестовой выборки, ни подмешивание случайных значений существенно не изменяют результат, который с учетом человеческого фактора довольно высок


## Задание 10
Загрузить jupyter notebook с решение на github и прислать ссылку

Выполнено